# Bloc 1 - Construction et alimentation d'une infrastructure de gestion de données - Plan your trip with Kayak

## Introduction

Kayak is a metasearch engine founded in 2004. The Kayak's website and mobile apps are available in about 30 countries and 20 languages to help travelers making decisions about their future trips.

### Problematic

Kayak discovered that their users would like to have more reliable information about their destination.

The company marketing team would like to create an application based on the weather and on the hotels in the area, which would recommend the best destinations at any given time.

### Scope

The marketing team wants to focus on the top-35 places to visit in France, which are: Le Mont-Saint-Michel, Saint-Malo, Bayeux, Le Havre, Rouen, Paris, Amiens, Lille, Strasbourg, Le Château du Haut Koenigsbourg, Colmar, Eguisheim, Besancon, Dijon, Annecy, Grenoble, Lyon, Les Gorges du Verdon, Bormes-les-Mimosas, Cassis, Marseille, Aix-en-Provence, Avignon, Uzes, Nîmes, Aigues-Mortes, Saintes-Maries-de-la-mer, Collioure, Carcassonne, l'Ariège, Toulouse, Montauban, Biarritz, Bayonne, La Rochelle.

### Aim and objectives

Overall aim: Get weather and hotel data for these cities and make it available for the marketing team.

Objectives:
- 1 - Get gps coordinates for each destination.
- 2 - Get weather data for each destination.
- 3 - Get hotels' information for each destination.
- 4 - Store all the information in a data lake.
- 5 - Extract, transform and load cleaned data from the data lake to a data warehouse.
- 6 - Provide a map of all destinations with weather information.
- 7 - Provide maps of top-5 destinations with top-20 hotels.

##
## Methods

### 1 - Library import and keys

An API key was required to get data from openweathermap.org. Do not forget to input your own key if you wish to re-use the following code. The same goes for the access to your AWS account and to your database.

### 2 - Gps coordinates

The Kayak team asked for data about the top-35 cities to visit in France. However the provided destination list contained places that were not cities per se (i.e. Gorges du Verdon). Therefore the list was slightly modified to replace these few places by the closest or biggest city in the area. City names were corrected to fit the French ortograph.

The gps coordinates (latitude and longitude) were extracted for the 35 cities with the API nominatim.org. A search by city name was used.

### 3 - Weather forecast

For each city, the weather data was extracted with openweathermap.org. The 5 days weather forecast API was chosen to obtain weather forecast from geographic coordinates for the 5 coming days with data every 3 hours. From the bunch of data that was available, the temperature (in degrees Celsius) and the probality of rain were selected as the most relevant to further choose top destinations based on the weather. Data was saved locally as a .csv file for future use (cnm_bloc1_data1_weather.csv).

### 4 - Hotels

For each city, accomodation data was scrapped from Booking.com. Since Kayak did not provide any criteria for hotel selection, the top-25 picks proposed by Booking.com were selected. Since some of the destinations are very small cities with few options for accomodation, the decision was made to not restrict the search to hotels but to also include other types of property such as bed and breakfasts or guesthouses in order to get the 20 accomodation options that were demanded. For the same reasons, nearby cities were also included.

The scraping was performed in two steps. First, a list of urls corresponding to accomodation web pages (on Booking.com) was obtained and saved locally as .json file (cnm_bloc1_data2_urls.json). Then, information on each accomodation was scrapped from the accomodation pages. This way, the following information was gathered for each accomodation: name, address, Booking.com url, gps coordinates, score given by users, and text description. Data was saved locally as a .json file (cnm_bloc1_data3_hotels.json).

### 5 - Data compilation and cleaning

For all cities, data about weather and accomodations were loaded from the corresponding files and compiled in a single dataframe. 

Data was then cleaned as follow: correction of city names, fixing of formatting issues, dropping of accomodations for which no score was available, and selection of the top-20 accomodations per city based on user score.

Data cleaning was checked and data was saved locally (cnm_bloc1_data4_final_dataset.csv) for future storage in a S3 bucket.

### 6 - Maps

The 35 cities were indicated on a map of France, with color indicating the probability of rain and size indicating the temperature forecasted for the 5 coming days.

The top-5 cities were selected as those with the smallest probability of rain and the highest temperature. For each of them, a map with the location of the top-20 accomodations was provided.

### 7 - Data storage and ETL

Final cleaned data was stored in a S3 bucket as a .csv file, extracted from the S3 bucket, transformed into a dataframe, and loaded to a SQL database.

##
## Conclusion

The request of the marketing team to collect data from the web about weather and accomodations for the top-35 destinations in France was fulfilled.

The cleaned collected data was transferred as a .csv file to a S3 bucket to then feed a SQL database.

##
## Code

### 1 - Library import and keys

In [ ]:
### 1 - library import and keys - import libraries ### ----

import pandas as pd
import numpy as np

import requests
import json

import boto3
from sqlalchemy import create_engine, text

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
### 1 - library import and keys - input keys ### ----

# WARNING! input your key for openweathermap.org here
key_openweather = "KEY"

# WARNING! input your credentials for aws
aws_access_key_id="ACCESS_KEY"
aws_secret_access_key="SECRET_ACCESS_KEY"

# WARNING! input your access values to connect to your database
dbuser = "USERNAME"
dbpass = "PASSWORD"
bdhost = "HOST_FROM_AMAZON_RDS"
dbname = "DBNAME"


###
### 2 - Gps coordinates

In [ ]:
### 2 - gps coordinates - set destination list ### ----

# set list of cities
cities = ["Le Mont-Saint-Michel", "Saint-Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
"Strasbourg", "Orschwiller", "Colmar", "Eguisheim", "Besançon", "Dijon", "Annecy", "Grenoble", "Lyon",
"Moustiers-Sainte-Marie", "Bormes-les-Mimosas", "Cassis", "Marseille", "Aix-en-Provence", "Avignon", "Uzès",
"Nîmes", "Aigues-Mortes", "Saintes-Maries-de-la-Mer", "Collioure", "Carcassonne", "Foix", "Toulouse",
"Montauban", "Biarritz", "Bayonne", "La Rochelle"]


In [ ]:
### 2 - gps coordinates - get coordinates ### ----

# initialise variable to store information on cities
data1 = pd.DataFrame(index = range(0,len(cities)), columns = ["city_id", "city_name", "city_latitude",
    "city_longitude"])

# get and store latitude and longitude from the API nominatim.org
for i in range(0,len(cities)):

    # get data
    city_current = cities[i]
    data_current =  requests.get("https://nominatim.openstreetmap.org/search?city=" + city_current + "&format=json")
    data_current = data_current.json()

    # store information
    data1.loc[i,"city_id"] = i+1
    data1.loc[i,"city_name"] = city_current
    data1.loc[i,"city_latitude"] = data_current[0]["lat"]
    data1.loc[i,"city_longitude"] = data_current[0]["lon"]
    

###
### 3 - Weather forecast

In [ ]:
### 3 - weather forecast - get data ### ----

# copy data for safety
data2 = data1.copy()

# initialize columns to store mean temperature and mean probability of precipitation
data2["city_temperature"] = np.nan
data2["city_precipitation"] = np.nan

# get and store weather data from the API openweathermap.org
for i in range(0,data1.shape[0]):

    # get data
    data_current =  requests.get("https://api.openweathermap.org/data/2.5/forecast?lat=" + 
        str(data2.loc[i,"city_latitude"]) + "&lon=" + str(data2.loc[i,"city_longitude"]) + 
        "&appid=" + key_openweather + "&units=metric")
    data_current = data_current.json()

    # initialise temporary variables
    temperature_temp = []
    precipitation_temp = []

    # store information for 5 next days
    for j in range(0,len(data_current["list"])):
        temperature_temp.append(data_current["list"][j]["main"]["temp"])
        precipitation_temp.append(data_current["list"][j]["pop"])
    
    # store summary data
    data2.loc[i,"city_temperature"] = np.mean(temperature_temp)
    data2.loc[i,"city_precipitation"] = np.mean(precipitation_temp)


In [ ]:
### 3 - weather forecast - save data ### ----

# rename and save data
data2.to_csv("cnm_bloc1_data1_weather.csv", index = False)


###
### 4 - Hotels

In [ ]:
### 4 - hotels - get booking.com urls for hotels in each city ### ----

# run python script to get and save urls
!python cnm_bloc1_scraping1.py


In [ ]:
### 4 - hotels - get hotel info for each city ### ----

# run python script to get and save hotel info
!python cnm_bloc1_scraping2.py


###
### 5 - Data compilation and cleaning

In [ ]:
### 5 - data compilation and cleaning - compile data ### ----

# open saved files and get data
data_weather = pd.read_csv("cnm_bloc1_data1_weather.csv")
file_urls = open("cnm_bloc1_data2_urls.json")
file_urls = json.load(file_urls)
data_urls = pd.DataFrame(file_urls)
file_hotels = open("cnm_bloc1_data3_hotels.json")
file_hotels = json.load(file_hotels)
data_hotels = pd.DataFrame(file_hotels)

# initialize dataframe to fit city and weather info to hotel info
data_cities = pd.DataFrame(np.NaN, index = range(0,data_hotels.shape[0]), columns = data_weather.columns)

# get unique cities
cities_unique = data_weather["city_name"].unique()

# fill new dataframe
for city in cities_unique:

    # get masks for current city
    mask1 = data_hotels["city_name"] == city
    mask2 = data_weather["city_name"] == city
    
    # fill with weather data
    data_cities.loc[mask1,"city_id"] = data_weather.loc[mask2,"city_id"].values[0]
    data_cities.loc[mask1,"city_name"] = data_weather.loc[mask2,"city_name"].values[0]
    data_cities.loc[mask1,"city_latitude"] = data_weather.loc[mask2,"city_latitude"].values[0]
    data_cities.loc[mask1,"city_longitude"] = data_weather.loc[mask2,"city_longitude"].values[0]
    data_cities.loc[mask1,"city_temperature"] = data_weather.loc[mask2,"city_temperature"].values[0]
    data_cities.loc[mask1,"city_precipitation"] = data_weather.loc[mask2,"city_precipitation"].values[0]

# drop city name in data_hotels and compile data
data_hotels = data_hotels.drop(["city_name"], axis = 1)
data_final = pd.concat([data_cities, data_hotels], axis = 1)


In [ ]:
### 5 - data compilation and cleaning - clean data ### ----

# part 1 - fix inconsistencies in city names

# get index of rows with missing values in city info
index_nan = data_final.loc[data_final["city_name"].isnull(),:].index

# get columns with missing values
columns_nan = data_weather.columns

# loop through rows
for i in index_nan:

    # get url of the hotel
    url_current = data_final.loc[i,"hotel_url"]

    # get city corresponding to that url
    city_current = data_urls.loc[data_urls["url"] == url_current,"city"].values[0]

    # fill dataframe with info from data_weather
    data_final.loc[i,columns_nan] = data_weather.loc[data_weather["city_name"] == city_current,:].values[0]


# part 2 - fix formatting issues

# fix hotel score 
data_final.loc[data_final["hotel_score"].isnull(),"hotel_score"] = np.NaN
data_final["hotel_score"] = data_final["hotel_score"].str.replace(",",".")
data_final.loc[data_final["hotel_score"].notnull(),"hotel_score"] = ["".join(char for char in score 
    if char in set("0123456789.")) for score in data_final["hotel_score"] if type(score) == str]
data_final.loc[data_final["hotel_score"] == "","hotel_score"] = np.NaN
data_final["hotel_score"] = data_final["hotel_score"].astype(float)

# fix hotel latitude and longitude
data_final["hotel_latitude"] = data_final["hotel_latitude"].astype(float)
data_final["hotel_longitude"] = data_final["hotel_longitude"].astype(float)

# strip brackets in hotel descriptions (stored as lists)
data_final["hotel_description"] = [desc[0] for desc in data_final["hotel_description"]]


# part 3 - drop rows containing missing values in hotel score and select top-20 accomodations

# drop accomodation if no score available
index_drop = data_final.loc[data_final["hotel_score"].isnull(),:].index
data_final = data_final.drop(index_drop, axis = 0)

# get unique cities
cities_unique = data_final["city_name"].unique()

# initialise index to store index of accomodations to keep
index_keep = [] 

# loop through cities
for city in cities_unique:

    # get data for current city
    data_current = data_final.loc[data_final["city_name"] == city,:]

    # sort by score and get index
    index_current = list(data_current.sort_values("hotel_score", ascending = False).index)
    index_keep += index_current[0:20]

# select top-20 per city based on score
data_final = data_final.loc[index_keep,:]


In [ ]:
### 5 - data compilation and cleaning - check cleaning ### ----

# print shape of data
print("Number of rows: {}".format(data_final.shape[0]))
print("Number of columns: {}".format(data_final.shape[1]))
print()

# display dataset
pd.set_option('display.max_columns', None)
print("Dataset display: ")
display(data_final.head())
print()

# check wether some columns are full of NaNs
column_nan_full = data_final.columns[data_final.isnull().all()]
column_nb = len(column_nan_full)

# get percentage of missing values in columns
percent_nan_col = data_final.isnull().sum() / data_final.shape[0] * 100

# print report
print("COLUMNS")
print("{} columns out of {} are fully filled with missing values".format(column_nb,data_final.shape[1]))
print("Percentage of missing values per column:\n{}".format(percent_nan_col))


In [ ]:
### 5 - data compilation and cleaning - save data ### ----

# rename and save data
data_final.to_csv("cnm_bloc1_data4_final_dataset.csv", index = False)


###
### 6 - Maps

In [ ]:
### 6 - maps - get data ### ----

# load clean dataset
data_plot = pd.read_csv("cnm_bloc1_data4_final_dataset.csv")


In [ ]:
### 6 - maps - plot destination overview ### ----

# get unique cities
cities_unique = data_plot["city_name"].unique()

# create dataframe to store unique data
data_fig1 = pd.DataFrame(index = range(0,len(cities_unique)), 
    columns = ["city","temperature","rain_proba","latitude","longitude"])

# get data for each city
for i in range(0, data_fig1.shape[0]):

    # get current city
    city_current = cities_unique[i]

    # get current data
    data_current = data_plot.loc[data_plot["city_name"] == city_current,:].reset_index(drop = True)

    # fill dataframe
    data_fig1.loc[i,"city"] = city_current
    data_fig1.loc[i,"latitude"] = data_current.loc[0,"city_latitude"]
    data_fig1.loc[i,"longitude"] = data_current.loc[0,"city_longitude"]
    data_fig1.loc[i,"temperature"] = data_current.loc[0,"city_temperature"]
    data_fig1.loc[i,"rain_proba"] = data_current.loc[0,"city_precipitation"]

data_fig1["temperature"] = data_fig1["temperature"].astype(float)
data_fig1["rain_proba"] = data_fig1["rain_proba"].astype(float)

# plot destinations
fig1 = px.scatter_mapbox(
    data_fig1, 
    lat = "latitude", 
    lon = "longitude", 
    color = "rain_proba",
    size = "temperature",
    size_max = 15,
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig1.update_layout(
        title_text = "Figure 1. Overview of destinations",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig1.update_mapboxes(
    style = "carto-positron",
    zoom = 4)

fig1.show()


In [ ]:
### 6 - maps - plot top-5 destination ### ----

# get top-5 destinations based on rain probability
data_fig1["rain_proba"] = data_fig1["rain_proba"].round(2)
data_fig1["temperature"] = data_fig1["temperature"].round(1)
data_fig1 = data_fig1.sort_values(["rain_proba","temperature"], ascending = [True, False]).reset_index(drop = True)
cities_top = data_fig1.loc[0:5,"city"].values

# extract data
data_fig2 = data_plot.loc[data_plot["city_name"].isin(cities_top),:]

# plot top-1 destination
fig21 = px.scatter_mapbox(
    data_fig2.loc[data_fig2["city_name"] == cities_top[0]], 
    lat = "hotel_latitude", 
    lon = "hotel_longitude", 
    color = "hotel_score",
    size = [1] * data_fig2.loc[data_fig2["city_name"] == cities_top[0]].shape[0],
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig21.update_layout(
        title_text = "Figure 2-1. Recommended hotels in " + cities_top[0],
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig21.update_mapboxes(
    style = "carto-positron",
    zoom = 11)

# plot top-2 destination
fig22 = px.scatter_mapbox(
    data_fig2.loc[data_fig2["city_name"] == cities_top[1]], 
    lat = "hotel_latitude", 
    lon = "hotel_longitude", 
    color = "hotel_score",
    size = [1] * data_fig2.loc[data_fig2["city_name"] == cities_top[1]].shape[0],
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig22.update_layout(
        title_text = "Figure 2-2. Recommended hotels in " + cities_top[1],
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig22.update_mapboxes(
    style = "carto-positron",
    zoom = 11)

# plot top-3 destination
fig23 = px.scatter_mapbox(
    data_fig2.loc[data_fig2["city_name"] == cities_top[2]], 
    lat = "hotel_latitude", 
    lon = "hotel_longitude", 
    color = "hotel_score",
    size = [1] * data_fig2.loc[data_fig2["city_name"] == cities_top[2]].shape[0],
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig23.update_layout(
        title_text = "Figure 2-3. Recommended hotels in " + cities_top[2],
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig23.update_mapboxes(
    style = "carto-positron",
    zoom = 11)

# plot top-4 destination
fig24 = px.scatter_mapbox(
    data_fig2.loc[data_fig2["city_name"] == cities_top[3]], 
    lat = "hotel_latitude", 
    lon = "hotel_longitude", 
    color = "hotel_score",
    size = [1] * data_fig2.loc[data_fig2["city_name"] == cities_top[3]].shape[0],
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig24.update_layout(
        title_text = "Figure 2-4. Recommended hotels in " + cities_top[3],
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig24.update_mapboxes(
    style = "carto-positron",
    zoom = 11)

# plot top-5 destination
fig25 = px.scatter_mapbox(
    data_fig2.loc[data_fig2["city_name"] == cities_top[4]], 
    lat = "hotel_latitude", 
    lon = "hotel_longitude", 
    color = "hotel_score",
    size = [1] * data_fig2.loc[data_fig2["city_name"] == cities_top[4]].shape[0],
    color_continuous_scale = px.colors.diverging.Portland)

# update layout
fig25.update_layout(
        title_text = "Figure 2-5. Recommended hotels in " + cities_top[4],
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)
fig25.update_mapboxes(
    style = "carto-positron",
    zoom = 11)

fig21.show()
fig22.show()
fig23.show()
fig24.show()
fig25.show()


###
### 7 - Data storage and ETL

```python
### 7 - data storage and etl - data storage ### ----

# final cleaned data (cnm_bloc1_data4_final_dataset.csv) stored in a s3 bucket

```
![alt text](cnm_bloc1_s3_bucket.png "S3 bucket with cleaned data")

```python

# database instance in rds 

```
![alt text](cnm_bloc1_sql_database.png "RDS database instance")


In [ ]:
### 7 - data storage and etl - etl ### ----

# extract data from s3 bucket
session = boto3.Session(aws_access_key_id, aws_secret_access_key)
s3 = boto3.client('s3')
s3.download_file('cnm-bloc1', 'cnm_bloc1_data4_final_dataset.csv', 'my_file.csv')

# transform into dataframe
my_data = pd.read_csv("my_file.csv")

# load to database
engine = create_engine("postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}", echo = True)
my_data.to_sql("my_final_table", engine)
